In [1]:
import random
import numpy as np

import matplotlib.pyplot as plt
import plotly.graph_objects as go
import plotly.figure_factory as ff

from functions import *

In [27]:
# задаем число маяков
nanchors = 4

# число углов комнаты
ncorners = 4

# шаг сетки
stepx = 0.1
stepy = 0.1

# массив координат по х и у
# xcoord = np.array((4.5, 6.5, 3.5, 1.5))
# ycoord = np.array((1.5, 2.5, 6.5, 3.5))

xcoord = np.array((4.5, 4.5, 3.5, 1.5))
ycoord = np.array((1.5, 2.5, 6.5, 3.5))

# xcoord = np.array((0.5, 3.5, 3.5, 0.5))
# ycoord = np.array((0.5, 0.5, 3.5, 3.5))

In [28]:
# формируем комнату
room = createRoom(xcoord, ycoord)
# формируем точки внутри помещения
pointsIn, pointsOut = createPoints(stepx, stepy, xcoord, ycoord)

# нормировка комнаты
roomScaler, pointsInScaler, pointsOutScaler, scaler = standardScalerPoints(xcoord, ycoord, pointsIn, pointsOut)
# нормировка по координатам
xcoordScaler = roomScaler[:, 0]
ycoordScaler = roomScaler[:, 1]

In [29]:
# генерируем координаты маяков
# можно не сравнивать со всеми точками внутри - т.к. места маяков не попадают на сетку
coordAnchorsScaler = createCoordAnchors(nanchors, xcoordScaler, ycoordScaler)

# подумать нужны ли те точки, которые мы хотим видеть pointsVisible
DOP = estimateDOP(roomScaler, pointsInScaler, coordAnchorsScaler, nanchors, DOPmax=20)

In [30]:
plotRoom(roomScaler, pointsInScaler, pointsOutScaler)
plotAnchors(coordAnchorsScaler, roomScaler)
plotDOPfactor(roomScaler, coordAnchorsScaler, DOP, pointsInScaler)

In [31]:
polyCoordAnchorsScaler = polyCoordAnchors(xcoordScaler, ycoordScaler, roomScaler, coordAnchorsScaler)

In [32]:
DOP = estimateDOP(roomScaler, pointsInScaler, polyCoordAnchorsScaler, nanchors, DOPmax=20)

In [33]:
plotAnchors(polyCoordAnchorsScaler, roomScaler)
plotDOPfactor(roomScaler, polyCoordAnchorsScaler, DOP, pointsInScaler)

In [34]:
npopul = 100     # Размер популяции
nsurv = 20       # Количество выживших (столько лучших переходит в новую популяцию)
nnew = npopul - nsurv # Количество новых (столько новых ботов создается)
l = nanchors * 2         # Длина бота
epohs = 20   # Количество эпох

mut = 0.3       # Коэфициент мутаций

popul = []       # Двумерный массив популяции, размерностью [n, l]. 100 ботов по 8 компонентов каждый
val = []         # Одномерный массив значений этих ботов

# Проходим по всей длине популяции
for i in range(npopul):

    bot = createCoordAnchors(nanchors, xcoordScaler, ycoordScaler)
    bot = polyCoordAnchors(xcoordScaler, ycoordScaler, roomScaler, bot)
    popul.append(bot)

    # Пробегаемся по всем эпохам
for it in range(epohs):

    # Создаем пустой список для значений ботов
    val = []

    # Проходим по всей длине популяции
    for bot in popul:
        # Берем очередного бота
        DOP = estimateDOP(roomScaler, pointsInScaler, bot, nanchors, DOPmax=20)
        val.append(sum(DOP) / len(DOP))

        # Получаем новую популяцию и сортированный список значнией
    newpopul, sval = getSurvPopul(popul, val, nsurv, 0)
    # Выводим 5 лучших ботов
    print(it, " ", [round(s,8) for s in sval[0:5]])
    if it % 5 == 0:
        plotAnchors(newpopul[0], roomScaler)

    # Проходимся в цикле nnew-раз
    for i in range(nnew):

        # Из newpopul(новой популяции) получаем двух случайных родителей-ботов
        botp1, botp2 = getParents(newpopul, nsurv)
        # Массив для нового бота
        newbot = []

        # проходимся по длине бота и осуществляем смешивание/скрещивание от родителей
        for j in range(l):

            # Получаем значение для j-ого компонента бота
            x = crossPointFrom2Parents(botp1.reshape(l), botp2.reshape(l), j)
            # Добавляем к значению бота случайную величину, зависящую от коэфециента мутации
            x += mut*(2*random.random() - 1.0)
            # Добавляем новое значение в бота
            newbot.append(x) # Добавялем новое значение в бота

        # Добавляем бота в новую популяцию
        newbot = np.array(newbot).reshape((nanchors,2))
        newbot = polyCoordAnchors(xcoordScaler, ycoordScaler, roomScaler, newbot)
        newpopul.append(newbot)

        #(таким образом к nsurv-лучших ботов предыдующей популяции добавится nnew-новых ботов)

    # Записываем в popul посчитанную новую популяцию
    popul = newpopul

0   [1.09208283, 1.11214975, 1.11919334, 1.12093187, 1.12136829]


1   [1.07978911, 1.09208283, 1.09497408, 1.09958033, 1.10602987]
2   [1.07978911, 1.08428892, 1.08827322, 1.09208283, 1.09497408]
3   [1.06465708, 1.07978911, 1.08428892, 1.08827322, 1.09208283]
4   [1.06465708, 1.07457291, 1.07978911, 1.08334099, 1.08424836]
5   [1.06465708, 1.07045445, 1.07457291, 1.07978911, 1.08334099]


6   [1.05648288, 1.06465708, 1.06827506, 1.07045445, 1.07170247]
7   [1.05648288, 1.06465708, 1.06827506, 1.07045445, 1.07170247]
8   [1.05648288, 1.06465708, 1.06612823, 1.06784554, 1.06827506]
9   [1.05648288, 1.06226406, 1.06465708, 1.06612823, 1.06784554]
10   [1.0491978, 1.05648288, 1.06226406, 1.06465708, 1.06612823]


11   [1.0491978, 1.05381554, 1.05648288, 1.05932946, 1.06032729]
12   [1.0491978, 1.05351148, 1.05381554, 1.05555323, 1.05648288]
13   [1.0491978, 1.05351148, 1.05381554, 1.05555323, 1.0560029]
14   [1.0491978, 1.05351148, 1.05381554, 1.05555323, 1.0560029]
15   [1.0491978, 1.05351148, 1.05381554, 1.05555323, 1.05579043]


16   [1.0491978, 1.05163476, 1.05351148, 1.05381554, 1.05555323]
17   [1.0491978, 1.05163476, 1.05351148, 1.05381554, 1.05503254]
18   [1.04879928, 1.0491978, 1.04947064, 1.05118409, 1.05163476]
19   [1.04879928, 1.0491978, 1.04947064, 1.05118409, 1.05155726]
